# LOAD THE X_TRAIN_WOE AND Y_TRAIN DATASETS

In [1]:
import pandas as pd
import numpy as np

In [2]:
x_train_woe=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_woe.csv")
y_train=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\y_train.csv")

# FIT THE MODEL WITH ALL THE 10 VARIABLES

In [3]:
import statsmodels.api as sm

In [4]:
#add constant
x=sm.add_constant(x_train_woe)

In [5]:
#Fit the model
model=sm.Logit(y_train,x).fit()

Optimization terminated successfully.
         Current function value: 0.252544
         Iterations 7


In [6]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 TARGET   No. Observations:               246008
Model:                          Logit   Df Residuals:                   245997
Method:                           MLE   Df Model:                           10
Date:                Sat, 21 Feb 2026   Pseudo R-squ.:                 0.09981
Time:                        08:47:15   Log-Likelihood:                -62128.
converged:                       True   LL-Null:                       -69017.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -2.4355      0.008   -299.795      0.000      -2.451      -2.420
EXT_SOURCE_3           -0.8509      0.014    -60.286      0.000      -0.879      -0.823
EXT_SOURCE_2           -0.8190      0.014    -57.870      0.000      -0.847      -0.791
EXT_SOURCE_1           -0.5460      0.023    -23.413      0.000      -0.592      -0.500
DAYS_EMPLOYED          -0.3645      0.029    -12.639      0.000      -0.421      -0.308
AMT_GOODS_PRICE        -0.6541      0.025    -25.744      0.000      -0.704      -0.604
OCCUPATION_TYPE        -0.3214      0.030    -10.859      0.000      -0.379      -0.263
DAYS_BIRTH              0.0850      0.034      2.529      0.011       0.019       0.151
ORGANIZATION_TYPE      -0.4276      0.037    -11.415      0.000      -0.501      -0.354
NAME_INCOME_TYPE       -0.1144      0.042     -2.709      0.007      -0.197      -0.032
NAME_EDUCATION_TYPE    -0.6099      0.037    -16.488      0.000      -0.682      -0.537
=======================================================================================
"""

**Inferences after fitting model with 10 variables**
1) Sign changed for DAYS_BIRTH, meaning variable is unstable
2) p-value for all the variables <0.05 ie. significant variables

## REFIT THE MODEL AFTER DROPPING DAYS_BIRTH

In [7]:
x_train_iter2=x_train_woe.drop("DAYS_BIRTH",axis=1)

In [8]:
x=sm.add_constant(x_train_iter2)

In [9]:
model_itr2=sm.Logit(y_train,x).fit()

Optimization terminated successfully.
         Current function value: 0.252557
         Iterations 7


In [10]:
model_itr2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 TARGET   No. Observations:               246008
Model:                          Logit   Df Residuals:                   245998
Method:                           MLE   Df Model:                            9
Date:                Sat, 21 Feb 2026   Pseudo R-squ.:                 0.09977
Time:                        08:59:23   Log-Likelihood:                -62131.
converged:                       True   LL-Null:                       -69017.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -2.4354      0.008   -299.825      0.000      -2.451      -2.419
EXT_SOURCE_3           -0.8476      0.014    -60.300      0.000      -0.875      -0.820
EXT_SOURCE_2           -0.8168      0.014    -57.812      0.000      -0.845      -0.789
EXT_SOURCE_1           -0.5306      0.022    -23.621      0.000      -0.575      -0.487
DAYS_EMPLOYED          -0.3423      0.027    -12.469      0.000      -0.396      -0.288
AMT_GOODS_PRICE        -0.6506      0.025    -25.643      0.000      -0.700      -0.601
OCCUPATION_TYPE        -0.3236      0.030    -10.944      0.000      -0.382      -0.266
ORGANIZATION_TYPE      -0.4154      0.037    -11.197      0.000      -0.488      -0.343
NAME_INCOME_TYPE       -0.0896      0.041     -2.186      0.029      -0.170      -0.009
NAME_EDUCATION_TYPE    -0.6236      0.037    -17.035      0.000      -0.695      -0.552
=======================================================================================
"""

**Inferences after fitting model with 9 variables(dropped DAYS_BIRTH)**
1) No sign flips, stable variables
2) p-value <0.05 significant variables

Now, we check multicolinearity

# CHECK FOR MULTICOLINEARITY WITHIN THE 9 REMAINING VARIABLES

In [17]:
def calculate_vif(dataframe, remove_cols=None):
    if remove_cols:
        dataframe.drop(remove_cols,axis=1,inplace=True)
    columns=dataframe.columns.tolist()
    for col in columns:
        y=dataframe[col]
        x=dataframe.drop(col,axis=1)
        x=sm.add_constant(x)
        model_linear=sm.OLS(y,x).fit()
        r_squ=model_linear.rsquared
        vif=1/(1-r_squ)
        print(f"VIF for {col} is : {vif}")

In [ ]:
# for col in columns:
#     y=col
#     x=x_train_iter2.drop(col,axis=1)
#     x=sm.add_constant(x)
#     model_linear=sm.OLS(y,x).fit()
#     r_squ=model_linear.rsquared
#     vif=1/(1-squ)
#     print(f"VIF for {col} is : {vif}")

In [18]:
#calculate vif for x_train_iter_2
calculate_vif(x_train_iter2)

VIF for EXT_SOURCE_3 is : 1.0423412045533504
VIF for EXT_SOURCE_2 is : 1.0530679563508591
VIF for EXT_SOURCE_1 is : 1.0751306710990833
VIF for DAYS_EMPLOYED is : 1.274107936343735
VIF for AMT_GOODS_PRICE is : 1.027279437705306
VIF for OCCUPATION_TYPE is : 1.3514111111905436
VIF for ORGANIZATION_TYPE is : 1.7782742245727312
VIF for NAME_INCOME_TYPE is : 1.664159406517825
VIF for NAME_EDUCATION_TYPE is : 1.1162860852469862


**Inference after calcuating VIF**
-No Multicolinearity
-VIF<5 for each variable 

# FINALIZE X_TRAIN AND X_TEST

In [20]:
x_train_final=x_train_iter2

In [19]:
x_test_woe=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_woe.csv")

In [21]:
x_test_woe.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,DAYS_BIRTH,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,-0.369300,0.342355,-0.011893,-0.374438,-0.044244,0.281957,0.266501,0.331053,0.366365,0.454906
1,0.796312,0.079621,-0.011893,0.571947,-0.116798,-0.285927,0.126310,-0.028921,-0.191389,-0.114665
2,-0.065417,0.224187,-0.049847,0.301072,-0.044244,-0.303906,-0.094726,-0.150859,-0.191389,-0.114665
3,0.956332,1.064822,-0.049847,0.571947,-0.557737,0.311292,0.266501,0.062787,0.088176,-0.114665
4,-0.037298,0.079621,-0.611732,-0.192015,-0.044244,-0.303906,0.004733,-0.434254,-0.191389,-0.114665


In [22]:
x_test_final=x_test_woe.drop("DAYS_BIRTH",axis=1)

# EXPORT X_TEST_FINAL AND X_TRAIN_FINAL

In [23]:
x_test_final.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_final.csv",index=False)
x_train_final.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_ttrain_final.csv",index=False)